This is the scrap the metadata of the story on webnovel.com

As of May 22, 2024: The remaining work is to collect Qidian reviews, replies, and user information and free accessible content of the stories (free chapter DONE).

As of May 29, 2024: Free Cha[etr collection is puy on hold. Comments and replies are beong collected. I use mostly my computer

In [3]:
import os
import re
import json
import time
import requests
import numpy as np
import pandas as pd
#from lxml import html
from tqdm import tqdm
from bs4 import BeautifulSoup as bs


In [4]:
bookList = pd.read_csv('../data/bookList.csv')
bookList = bookList.dropna(subset=['qidianUrl'])

In [5]:
bookList.head()

,qidianUrl,webnovelUrl,qidianBookId,webnovelBookId
0,https://www.qidian.com/book/1003692682/,https://www.webnovel.com/book/super-gene_80224...,1.003693e+09,8022472105002805
1,https://www.qidian.com/book/1010868264/,https://www.webnovel.com/book/lord-of-mysterie...,1.010868e+09,11022733006234505
2,https://www.qidian.com/book/1009480992/,https://www.webnovel.com/book/the-legendary-me...,1.009481e+09,8662546605001405
3,https://www.qidian.com/book/3471716/,https://www.webnovel.com/book/reincarnation-of...,3.471716e+06,8527113906000305
4,https://www.qidian.com/book/1004179514/,https://www.webnovel.com/book/library-of-heave...,1.004180e+09,6831850602000905


In [6]:
bookList.shape

(110, 4)

## Auxiliary Functions

In [19]:
def get_ListOfFreeChapters(bookId):
  '''This function returns the list of free chapters.'''
  url = "https://www.qidian.com/ajax/book/category"
  querystring = {"_csrfToken":"07cc5d3d-fb2b-4d08-8915-561aae1b4d86","bookId":bookId,"w_tsfp":"ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqshUZ+M1IV5v9fldCyCt5Mxutrd9MVxYnGHXtYmfBgSR8mYb5tH1VPHx8NlntdKRQJtA8/bUQRJde987TcQf24IIEK0jm4tIIZEnrVj2F0Ftnd337ZlCa8hbMFbxl0yufqB0Jtsez+Yyw6FRUDKI2EKfeCevf5z28MDtH2KgQSgeQVhAM5C1xbB0HwcWmV1sxPpae0JNRSwK8euXfA1qzKik2z2OJT40RA241Zl70Fme4Hh2kHJK3FFJwptd0Sg0+x6JOz2Z7o3syMZXKxPSVkVqQ8ZteI5+URPDSi9YHWPBfp6tQAARvJZ/9+wa3idg8m3Y00E+tc8xFljv5FXu2p3bjjzJYtcHDLJYCZdcYkAPprpMH4zWUJBEA=="}

  payload = ""
  headers = {
      "cookie": "supportWebp=true; newstatisticUUID=1716733315_1086707840; fu=1242577821",
      "Accept": "application/json, text/javascript, */*; q=0.01",
      "Accept-Language": "en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7",
      "Connection": "keep-alive",
      "Cookie": "supportWebp=true; _gid=GA1.2.1050708547.1716387806; supportwebp=true; Hm_lvt_f00f67093ce2f38f215010b699629083=1716387812,1716537935; _yep_uuid=aa73416d-c47a-e71f-b966-c9f20dcd42d3; _csrfToken=07cc5d3d-fb2b-4d08-8915-561aae1b4d86; qdrsnew=7%7C3%7C0%7C0%7C1; newstatisticUUID=1716733315_1086707840; fu=1597427940; traffic_utm_referer=; trkf=1; _gat_gtag_UA_199934072_2=1; Hm_lpvt_f00f67093ce2f38f215010b699629083=1716803179; _ga_FZMMH98S83=GS1.1.1716801424.19.1.1716803195.0.0.0; _ga=GA1.1.206926275.1716387806; _ga_PFYW0QLV3P=GS1.1.1716801424.19.1.1716803196.0.0.0; w_tsfp=ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqshUZ+M1IV5v9fldCyCt5Mxutrd9MVxYnGHXtYmfBgSR8mRb5tH1VPHx8NlntdKRQJtA8/bUQRJde987TcQf24IIEK0jm4tIIZEnrVj2F0Ftnd337ZlCa8hbMFbxl0yufqB0Jtsez+Yyw6FRUDKI2EKfeCevf5z28MDtH2KgQSgeQVhAM5C1xbB0HwcWmV1sxPpae0JNRSwK8euXfA1qzKik2z2OJT40RA241Zl70Fme4Hh2kHJK3FFJwptd0Sg0+x6JOz2Z7o3syMZXKxPSVkVqQ8ZteI5+URPDSi9YHWPBfp6tQAARvJZ/9+wa3idg8m3Y00E+tc8kg0zvpZRvT97MWn0KoteSmefYXJdft4EY5q8ZikxUkRBEA==",
      "Referer": "https://www.qidian.com/book/3471716/",
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }

  response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
  response_dict = json.loads(response.text)
  volumes = response_dict['data']['vs']
  freeChapterIds = []
  for volume in volumes:
    if volume['vS'] == 0:
      for ch in volume['cs']:
        freeChapterIds.append(ch['id'])
  return freeChapterIds


def get_Chapter(bookId,chapterId):
  chapterUrl = 	'https://www.qidian.com/chapter/' + bookId + '/' + chapterId
  response = requests.request("GET", chapterUrl, data=payload, headers=headers)
  soup = bs(response.text,'html')
  content = json.loads(soup.find_all('script',{'type':"application/json"})[0].text)
  content = content['pageContext']['pageProps']['pageData']['chapterInfo']['content'].replace('\u3000','').replace('<p>','')
  return content



In [20]:
#Comment Summary
def get_chapterCommentSummary(bookId,chapterId,referer):
  urlSummary = "https://www.qidian.com/ajax/chapterReview/reviewSummary"
  querystring = {"bookId":bookId,"chapterId":chapterId,"_csrfToken":"07cc5d3d-fb2b-4d08-8915-561aae1b4d86","w_tsfp":"ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiWpqF1YR+uNfldCyCt5Mxutrd9MVxYnGHUdUteREQQM6Vb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0y+OoJoni/oAamfRk9Frk0ghrNjlFLG3tX4BG7d+9UNxqlIdutXr5o/HO3l3uOPoWvrRUn4FYyuBc8C8D231qZbDQSRFQwMAangcokfP34M7Im7SxPDq4dVF1BqQcEves4/FlKCynsMSCOV/4uvQYSWuAIrZPzO3aUiojmMghZ6N0ulVs3u5IF7zhyZ2n3Ld5aSGHLZXMPeY0Aa5y7JjYgAhsGDmwPo0gENmpZXugibI6X7xC1KEpc1eMxbeTqf+AEaCjFAaLuDeA8CCPx"}

  payload = ""
  headers = {
      "cookie": "supportWebp=true; newstatisticUUID=1716733315_1086707840; fu=1242577821",
      "Accept": "application/json, text/plain, */*",
      "Accept-Language": "en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7",
      "Cookie": "supportWebp=true; _gid=GA1.2.1050708547.1716387806; supportwebp=true; Hm_lvt_f00f67093ce2f38f215010b699629083=1716387812,1716537935; _yep_uuid=aa73416d-c47a-e71f-b966-c9f20dcd42d3; _csrfToken=07cc5d3d-fb2b-4d08-8915-561aae1b4d86; qdrsnew=7%7C3%7C0%7C0%7C1; traffic_utm_referer=; trkf=1; newstatisticUUID=1716733315_1086707840; fu=1597427940; Hm_lpvt_f00f67093ce2f38f215010b699629083=1716737546; _gat_gtag_UA_199934072_2=1; _ga_PFYW0QLV3P=GS1.1.1716736383.17.1.1716738404.0.0.0; _ga=GA1.1.206926275.1716387806; _ga_FZMMH98S83=GS1.1.1716736383.17.1.1716738404.0.0.0; w_tsfp=ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiWpqF1YR+uNfldCyCt5Mxutrd9MVxYnGHUdUteREQQM2Tb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0y+OoJoni/oAamfRk9Frk0ghrNjlFLG3tX4BG7d+9UNxqlIdutXr5o/HO3l3uOPoWvrRUn4FYyuBc8C8D231qZbDQSRFQwMAangcokfP34M7Im7SxPDq4dVF1BqQcEves4/FlKCynsMSCOV/4uvQYSWuAIrZPzO3aUiojmMghZ6N0ulVs3u5IF7zhyZ2n3Ld5aSGHLZXMPeY0Aa5y7JjYgAhsGDmwPo0gENmpeC74qPNGcuBW3fUYKhLM7Y7HvLOwLbnnFA/fpDbNjCCPx",
      "Referer": referer,
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }

  response = requests.request("GET", urlSummary, data=payload, headers=headers, params=querystring)

  commentSummary = json.loads(response.text)
  commentSummary = pd.DataFrame(commentSummary['data']['list'])
  return commentSummary

#Comments
def get_segmentComments(bookId,chapterId,segmentId,referer):
  urlComments = "https://www.qidian.com/ajax/chapterReview/reviewList"
  page = "1"
  page_int = 1
  comments = []
  while True:
    querystring = {"bookId":bookId,"chapterId":chapterId,"page":page,"pageSize":"20","segmentId":segmentId,"type":"2","_csrfToken":"07cc5d3d-fb2b-4d08-8915-561aae1b4d86","w_tsfp":"ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiW52F2YF5t9fldCyCt5Mxutrd9MVxYnGHUdUsfhEdRsWZb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0y+OoJonigvBi/OVgCUeUdizzGwCtrD3gj4heydu9VNwmvJZ33Ca10+HGKliz0HpOo2BAm6Vk2uQs/BJWljmDfLAJFNQltJQnl1919Mfu+NZInuSQbWu8aHgRA90tg4fc6jUFaDy75KTXaEPkOtxUCQJ0KvI/6HXSeg8T5I3xZ7459kA80749TvTogem2jLdZHQGjKYG4Kf4xROsm6Zi5mWUVBQSUdtEtJfiUJV/kpe4ee6xe2K0Ne0rEzZOftKeQMaX3EUPa/BbY1XDG8+5Jm9wkYWa/7QolAOJkRCjeKhpusxNAhcoW221rmEXvStOxZKmLlAvBxe830x/kjvBDa"}
    payload = ""

    headers = {
        "cookie": "supportWebp=true; newstatisticUUID=1716733315_1086707840; fu=1242577821",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7",
        "Cookie": "supportWebp=true; _gid=GA1.2.1050708547.1716387806; supportwebp=true; Hm_lvt_f00f67093ce2f38f215010b699629083=1716387812,1716537935; _yep_uuid=aa73416d-c47a-e71f-b966-c9f20dcd42d3; _csrfToken=07cc5d3d-fb2b-4d08-8915-561aae1b4d86; qdrsnew=7%7C3%7C0%7C0%7C1; traffic_utm_referer=; trkf=1; newstatisticUUID=1716733315_1086707840; fu=1597427940; Hm_lpvt_f00f67093ce2f38f215010b699629083=1716739294; _ga_FZMMH98S83=GS1.1.1716736383.17.1.1716739293.0.0.0; _gat_gtag_UA_199934072_2=1; _ga=GA1.1.206926275.1716387806; _ga_PFYW0QLV3P=GS1.1.1716736383.17.1.1716739293.0.0.0; w_tsfp=ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiW52F2YF5t9fldCyCt5Mxutrd9MVxYnGHUdUsfhEdRsSWb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0y+OoJonigvBi/OVgCUeUdizzGwCtrD3gj4heydu9VNwmvJZ33Ca10+HGKliz0HpOo2BAm6Vk2uQs/BJWljmDfLAJFNQltJQnl1919Mfu+NZInuSQbWu8aHgRA90tg4fc6jUFaDy75KTXaEPkOtxUCQJ0KvI/6HXSeg8T5I3xZ7459kA80749TvTogem2jLdZHQGjKYG4Kf4xROsm6Zi5mWUVBQSUdtEtJfiUJV/kpe4ee6xe2K0Ne0rEzZOftKeQMaX3EUPa/BbY1XDG8+5Jm9wkYWa/7QolAOJkRCjeK25KtwYwncIayjg/gESqH5+0PITC/VvggLp+llPktuRDa",
        "Referer": referer,
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
    }

    response = requests.request("GET", urlComments, data=payload, headers=headers, params=querystring)
    #time.sleep(10*np.random.random())
    comments_dict = json.loads(response.text)
    if len(comments_dict['data']['list']) > 0:
      comments += comments_dict['data']['list']
      page_int += 1
      page = str(page_int)
    else:
      break
  return comments

In [21]:
def get_QidianMeta(url,bookId):
  payload = ""
  headers = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
      "Cookie": "_yep_uuid=02c2c6fd-521e-a2cd-6ac9-4f70e60b34aa; x-waf-captcha-referer=; newstatisticUUID=1713391240_558590204; _csrfToken=ynkbISkpAKNw2fcRba2COBIpyFXHMBCNiZYqBhfq; fu=310525340; _ga_FZMMH98S83=GS1.1.1713391243.1.0.1713391243.0.0.0; _gid=GA1.2.950724054.1713391244; _gat_gtag_UA_199934072_2=1; _ga_PFYW0QLV3P=GS1.1.1713391244.1.0.1713391244.0.0.0; _ga=GA1.1.1339574039.1713391244; traffic_utm_referer=; Hm_lvt_f00f67093ce2f38f215010b699629083=1713391245; Hm_lpvt_f00f67093ce2f38f215010b699629083=1713391245; w_tsfp=ltvgWVEE2utBvS0Q6KnqnUqsFTE7Z2R7xFw0D+M9Os09A6AoU5yB2Yd9u9fldCyCt5Mxutrd9MVxYnGCVd8kfxQcQMuVb5tH1VPHx8NlntdKRQJtA82OC18fJLsj7DRBKD9WLUOwjz4vI9VFyLNi2A8L5yon37ZlCa8hbMFbixsAqOPFm/97DxvSliPXAHGHM3wLc+6C6rgv8LlSgW2DugDuLi11A7pB1kyQ0yAdG3pV8w2pJbsDal7wcpK9Uv8wrTPzwjn3apCs2RYj4VA3sB49AtX02TXKL3ZEIAtrZUqukO18Lv3wdaN4qzsLCf4eS11Eql4QtrY++BJKCCu8ZSHbAPx7s1ECQ/Jb98m+NA=="
  }
  response = requests.request("GET", url, data=payload, headers=headers)
  soup = bs(response.text,'html')
  book_meta = {'bookId':bookId}

  qidian_meta = soup.find_all('meta')
  numberOfFreeChapters = len(get_ListOfFreeChapters(bookId))

  for meta in qidian_meta:
    if meta.get('property') in ['og:description','og:title','og:novel:category','og:novel:author','og:novel:author_link',
                                'og:novel:update_time','og:novel:latest_chapter_name','og:novel:latest_chapter_url']:
      book_meta.update({meta.get('property'):meta.get('content')})
    if meta.get('name') in ['keywords','']:
      book_meta.update({meta.get('name'):meta.get('content')})
    if meta.get('name') == 'description':
      book_meta.update({'numberOfChapters':re.search('[0-9]+',meta.get('content')).group()})
      book_meta.update({'numberOfFreeChapters':numberOfFreeChapters})

  genre = soup.find_all('a',{'data-eid':'qd_G10'})[0].get('title')
  book_meta.update({'genre':genre[:-2]})
  numberOfWords, totalRecommendations,numberOfWeeksRecommended = soup.find_all('p',{'class':'count'})[0].text.split(' ')
  book_meta.update({'numberOfWords':numberOfWords[:-1], 'totalRecommendations':totalRecommendations[:-3],'numberOfWeeksRecommended':numberOfWeeksRecommended[:-2]})
  return book_meta


## Collecting Meta Data

It only collects what we see on the page. I need to update the information on free chapters.

In [ ]:
allMeta = []
errors = []

for i,row in bookList.iterrows():
  url = row['qidianUrl']
  bookId = str(int(row['qidianBookId']))
  try:
    bookMeta = get_QidianMeta(url,bookId)
    allMeta.append(bookMeta)
  except:
    errors.append(url)
  time.sleep(30)


In [ ]:
print(errors)

In [ ]:
qidianMeta = pd.DataFrame(allMeta)
qidianMeta.to_csv('../data/qidianMeta_' +str(pd.Timestamp.today()) + '.csv',index=False)

In [ ]:
#correct number of free chapters in the meta file
bookIds = bookList['qidianBookId']
numberOfFreeChapters = []
for id in bookIds:
  id = str(int(id))
  with open('../data/qidianFreeChapterIds/' + id + '.txt',) as f:
      chapterIds = [b.strip() for b in f.readlines()]
  numberOfFreeChapters.append([id,len(chapterIds)])



In [ ]:
free_df = pd.DataFrame(numberOfFreeChapters,columns=['bookId','numberOfFreeChapters2'])
free_df['bookId'] = free_df['bookId'].astype('int')
meta_df = pd.read_csv('/content/drive/MyDrive/CITCalls/Ze/data/qidianMeta.csv')
meta_df['bookId'] = meta_df['bookId'].astype('int')
final_df = pd.merge(left=meta_df,right=free_df,left_on='bookId',right_on='bookId')
print(final_df.columns)
final_df = final_df.drop('numberOfFreeChapters',axis=1)
final_df = final_df.rename(columns={'numberOfFreeChapters2':'numberOfFreeChapters'})
new_column_order = ['bookId', 'keywords', 'numberOfChapters', 'numberOfFreeChapters','og:title', 'og:description',
       'og:novel:category', 'og:novel:author', 'og:novel:author_link',
       'og:novel:update_time', 'og:novel:latest_chapter_name',
       'og:novel:latest_chapter_url', 'genre', 'numberOfWords',
       'totalRecommendations', 'numberOfWeeksRecommended']
final_df = final_df[new_column_order]
print('________________')
print(final_df.columns)


Index(['bookId', 'keywords', 'numberOfChapters', 'numberOfFreeChapters',
       'og:title', 'og:description', 'og:novel:category', 'og:novel:author',
       'og:novel:author_link', 'og:novel:update_time',
       'og:novel:latest_chapter_name', 'og:novel:latest_chapter_url', 'genre',
       'numberOfWords', 'totalRecommendations', 'numberOfWeeksRecommended',
       'numberOfFreeChapters2'],
      dtype='object')
________________
Index(['bookId', 'keywords', 'numberOfChapters', 'numberOfFreeChapters',
       'og:title', 'og:description', 'og:novel:category', 'og:novel:author',
       'og:novel:author_link', 'og:novel:update_time',
       'og:novel:latest_chapter_name', 'og:novel:latest_chapter_url', 'genre',
       'numberOfWords', 'totalRecommendations', 'numberOfWeeksRecommended'],
      dtype='object')


In [ ]:
final_df.to_csv('../data/qidianMeta_onlyfreecolumn.csv',index=False)

## Free Chapters Content
There might be more chapters since the meta data was collected.

In [ ]:
for i,book in tqdm(bookList[49:53].iterrows()):
  bookUrl = book['qidianUrl']
  bookId = str(int(book['qidianBookId'])) # this is a variable in the data frame
  if bookUrl: # because some qidian urls are missing.
    chapters = get_listOfFreeChapters(bookId)
    print(len(chapters))
    free_content = open('../data/qidianFreeChapters/' + bookId + '.txt','w')
    ch_number = 1
    for chapterId in chapters:
      free_content.write(f"############## CHAPTER {ch_number} ##############" + '\n')
      free_content.write(get_Chapter(bookId,chapterId) + '\n')
      ch_number += 1
    free_content.close()
  else:
    pass

0it [00:00, ?it/s]

131


0it [01:53, ?it/s]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:

for bookId in tqdm(bookList['qidianBookId']):
  bookId = str(int(bookId))
  chapterIds = get_ListOfFreeChapters(bookId)
  with open('../data/qidianFreeChapterIds/' + bookId + '.txt', 'w') as f:
    for i in chapterIds:
      f.write(str(i) + '\n')

100%|██████████| 110/110 [01:43<00:00,  1.07it/s]


## Comments & Replies

In [33]:
metaList = os.listdir('../data/qidianFreeChapterMeta/')
metaList = [m.split('.')[0] for m in metaList]
bookList = [str(int(b)) for b in bookList['qidianBookId']]
missingIdList = [b for b in bookList if b not in metaList]
missingIdList

['1017141',
 '1019664125',
 '1010463703',
 '3447263',
 '3621897',
 '2248950',
 '1015421270',
 '1005406434',
 '1911245',
 '1003374314']

In [34]:
referer = 'www.google.com'
for bookId in tqdm(missingIdList):
  try:
    bookId = str(int(bookId))
    #print(bookId)
    summary_df = pd.DataFrame()
    with open('../data/qidianFreeChapterIds/' + bookId + '.txt', 'r') as f:
      chapterIds = [b.strip() for b in f.readlines()]
    for chapterId in chapterIds:
      temp_df = get_chapterCommentSummary(bookId,chapterId,referer)
      temp_df['qidianChapterId'] = chapterId
      summary_df = pd.concat([summary_df,temp_df],ignore_index=True)
    summary_df['qidianBookId'] = bookId
    summary_df.reset_index(drop=True)
    summary_df.to_csv('/content/drive/MyDrive/CITCalls/Ze/data/qidianFreeChapterMeta/' + bookId + '.csv',index=False)
    time.sleep(100) #sleep 5 minutes
  except ConnectionError:
    print(bookId, 'Connection Error. Waiting for a while!!')
    time.sleep(300) #sleep 5 minutes


100%|██████████| 10/10 [24:28<00:00, 146.88s/it]


In [ ]:
print('Starting collecting chapters!')
for chapterId in chapterIds[8:10]:
  print('Chapter:', chapterId)
  chapterId = str(chapterId)
  commentSummary = get_chapterCommentSummary(bookId,chapterId)
  print('Number of segments in this chapter:',len(commentSummary))
  chComments = []
  for segmentId in commentSummary['segmentId']:
    segmentId = str(segmentId)
    segmentComments = get_segmentComments(bookId,chapterId,segmentId)
    chComments += segmentComments
    time.sleep(50*np.random.random())
  comments_df = pd.DataFrame(chComments)
  print('Saving file',chapterId)
  comments_df.to_csv('../data/qidianReviews/' + bookId + '_' + chapterId + '.csv',index=False)
  time.sleep(50*np.random.random())


Starting collecting chapters!
Chapter: 341659059
Number of segments in this chapter: 29
Saving file 341659059
Chapter: 341660460
Number of segments in this chapter: 27
Saving file 341660460
Chapter: 341660505
Number of segments in this chapter: 30


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
comments_df.to_csv('../data/qidianReviews/' + bookId + '_' + chapterId + '.csv',index=False)

In [ ]:
# find comments with replies
# what if there are more than 20 replies
def get_Replies(reviewId):
  urlReplies = "https://www.qidian.com/ajax/chapterReview/quoteReviewList"
  idReplies = []
  querystring = {"reviewId":reviewId,"page":"1","pageSize":"20","_csrfToken":"07cc5d3d-fb2b-4d08-8915-561aae1b4d86","w_tsfp":"ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiW5uE2IF5udfldCyCt5Mxutrd9MVxYnGHUdUseBMURsmYb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0x6PMUol2JowKuayctQP5X0zPQgG5GWz9atQS4AOhbNBWsJ86sWuQ2qTX6yjj2aIWu31ZytgUis2o8F9fymWSdehUdal5+Ziyxl7wiFP3odLBB5mpMArlMPVkS+lwc4eFt5BIdD3qgZCGPDedyvAEFW/df/pz9LCqX0s7kJhpF+ox8ywR15sNZ/TJgZ2n3Ld5aSGHLZXMPeY0Aa5y7NCoyUUNTXTdM5hUWPHhKF/lwMdLNsFTzdlFUwLJiMO+/euELaXiXBae2BrMyDmK088NjsBRfW6OoENYCb4k="}

  payload = ""
  headers = {
      "cookie": "supportWebp=true; newstatisticUUID=1716733315_1086707840; fu=1242577821",
      "Accept": "application/json, text/plain, */*",
      "Accept-Language": "en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7",
      "Connection": "keep-alive",
      "Cookie": "supportWebp=true; _gid=GA1.2.1050708547.1716387806; supportwebp=true; Hm_lvt_f00f67093ce2f38f215010b699629083=1716387812,1716537935; _yep_uuid=aa73416d-c47a-e71f-b966-c9f20dcd42d3; _csrfToken=07cc5d3d-fb2b-4d08-8915-561aae1b4d86; qdrsnew=7%7C3%7C0%7C0%7C1; traffic_utm_referer=; trkf=1; newstatisticUUID=1716733315_1086707840; fu=1597427940; Hm_lpvt_f00f67093ce2f38f215010b699629083=1716739294; _ga_FZMMH98S83=GS1.1.1716736383.17.1.1716739293.0.0.0; _ga=GA1.1.206926275.1716387806; _ga_PFYW0QLV3P=GS1.1.1716736383.17.1.1716739293.0.0.0; w_tsfp=ltvgWVEE2utBvS0Q6KzqnEymFjk7Z2R7xFw0D+M9Os09BqQiW5uE2IF5udfldCyCt5Mxutrd9MVxYnGHUdUseBMURsiWb5tH1VPHx8NlntdKRQJtA5LbDQMZK+4h6TZDdTkMLBbmjWwvJIETxORl3lwJ5SAm37ZlCa8hbMFbxl0yufqB0Jtsez6fxRXUEnT7J2MGf/jJ9p0x6PMUol2JowKuayctQP5X0zPQgG5GWz9atQS4AOhbNBWsJ86sWuQ2qTX6yjj2aIWu31ZytgUis2o8F9fymWSdehUdal5+Ziyxl7wiFP3odLBB5mpMArlMPVkS+lwc4eFt5BIdD3qgZCGPDedyvAEFW/df/pz9LCqX0s7kJhpF+ox8ywR15sNZ/TJgZ2n3Ld5aSGHLZXMPeY0Aa5y7NCoyUUNTXTdM5hUWPHhKF/lwMdLNsFTzdlFUwOU1Nu/tfOwMbX3EU6S6AOQ0Djnu+Ztlsx0LDfOqEtUFb4k=",
      "Referer": "https://www.qidian.com/chapter/1003692682/341658919/",
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }

  response = requests.request("GET", urlReplies, data=payload, headers=headers, params=querystring)
  replies_dict = json.loads(response.text)
  idReplies += replies_dict['data']['list']
  return idReplies


In [ ]:
# Replies
# with the help of imsonia
# this commenst file should be the complete comments for a single book.

comments_df = pd.read_csv('../data/qidianReviews/1003692682_341658919.csv')
commentIdsWithReplies = comments_df['rootReviewId'].loc[comments_df['rootReviewReplyCount']!=0]
replies = []
for id in tqdm(commentIdsWithReplies):
  reviewId = str(id)
  replies += get_replies(reviewId)
  time.sleep(10*np.random.random())
replies_df = pd.DataFrame(replies)
replies_df.to_csv('../data/qidianReplies/' + bookId + '_' + chapterId + '.csv',index=False)

100%|██████████| 142/142 [14:23<00:00,  6.08s/it]


In [ ]:
replies_df.shape,comments_df.shape

((359, 32), (893, 32))

In [ ]:
replies_df

,reviewId,cbid,ccid,guid,userId,nickName,avatar,segmentId,content,status,...,userDislike,isSelf,essenceStatus,riseStatus,level,imagePre,imageDetail,rootReviewId,rootReviewReplyCount,ipAddress
0,610188832150913024,4939454803015703,15734505102665509,3575498121,348561317,白风荷,https://qidian.qpic.cn/qd_face/349573/7896346/100,-1,乍一看感觉有点奇怪，但仔细一想，还真挺有道理的哈,1,...,False,False,False,False,110,,,312261049028050944,0,内蒙古自治区
1,482139624450555904,4939454803015703,15734505102665509,120004682258,310588312,青_剥,https://qidian.qpic.cn/qd_face/349573/7343947/100,-1,好强♂,1,...,False,False,False,False,93,,,312261049028050944,0,未知
2,469482953676881920,4939454803015703,15734505102665509,2373767971,327700069,学海浮力小,https://qidian.qpic.cn/qd_face/349573/9703631/100,-1,当我打出这个问号，不是代表我有问题，而是我觉得你有问题,1,...,False,False,False,False,89,,,312261049028050944,0,未知
3,460273186051981312,4939454803015703,15734505102665509,2552723099,230803016,愿君多珍重,https://qidian.qpic.cn/qd_face/349573/12192238...,-1,了解了,1,...,False,False,False,False,85,,,312261049028050944,0,未知
4,457446167270391808,4939454803015703,15734505102665509,2691882438,220680704,田树举,https://qidian.qpic.cn/qd_face/349573/7990328/100,-1,巧了，我也是，打个卡就走。,1,...,False,False,False,False,84,,,312261049028050944,0,未知
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,278863457430863872,4939454803015703,15734505102665509,851001513528,225031241,西爱罗,https://qidian.qpic.cn/qd_face/349573/6115679/100,42,一刀999？那么这种刀，不知在哪里可以买得到呢？,1,...,False,False,False,False,15,None,None,278859619571400704,0,湖南
355,443948703771000832,4939454803015703,15734505102665509,1172198588,220509405,天下谁人不识服朕,https://qidian.qpic.cn/qd_face/349573/6182504/100,43,提示音是什么鬼？就不能是心灵暗语吗？,1,...,False,False,False,False,34,,,422165772144672768,0,未知
356,418181425364336640,4939454803015703,15734505102665509,119192365252,224566790,莫若离去,https://qidian.qpic.cn/qd_face/349573/3508680/100,43,看到这里就已经不想看了，写的什么玩意儿,1,...,False,False,False,False,31,,,202712940644794368,0,未知
357,249643736038965248,4939454803015703,15734505102665509,1793438418,210283384,弥阳星主,https://qidian.qpic.cn/qd_face/349573/11857410...,43,你看到后面就知道了,1,...,False,False,False,False,20,None,None,202712940644794368,0,上海


## User Profiles

Once all comments and replies are collected, I will start collecting user profiles.

In [64]:

#from the source code

def get_UserInfo(uderId):
    userUrl = 'https://my.qidian.com/user/' + userId
    header = { "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
'Content-Type': 'text/html; charset=utf-8',
'set-cookie':'csrfToken=yQltzgNj44UErVXluXQo0oDkcXXlzeJyBg1npj5F; expires=Fri, 06-Jun-2025 20:56:02 GMT; path=/; domain=.qidian.com',
'X-NWS-LOG-UUID': '5059556769341264456'}
    r = requests.get(userUrl,headers=header)
    soup = bs(r.text,'html')

    userInfo = soup.find_all('div',{'class':'header-msg'})[0]
    levelInfo = userInfo.find_all('h3',{'data-id':userId})[0].text
    genderInfo = userInfo.find_all('div',{'class':'header-msg-desc'})[0].text
    nameInfo = userInfo.find_all('div',{'class':'header-msg-title'})[0].text
    nameInfo = nameInfo.split(':')[-1].strip()
    numberOfFollowers = userInfo.find_all('span',{'class':"mr8"})[0].text
    numberOfFans = userInfo.find_all('span',{'class':"ml12 mr8"})[0].text

    pattern = r'\D*([0-9]*)\D+'
    favoriteBooks = soup.find_all('h2',{'class':"user-title"})[0].text
    numberOfFavoriteBooks = int(re.findall(pattern,favoriteBooks)[0])
    numberOfFavoriteBooks

    return [levelInfo,genderInfo,nameInfo,numberOfFollowers,numberOfFans,numberOfFavoriteBooks]

In [62]:
def get_UserHistory(userId):
    url = "https://my.qidian.com/ajax/User/FriendHistory"

    querystring = {"_csrfToken":"07cc5d3d-fb2b-4d08-8915-561aae1b4d86","id":userId}

    payload = ""
    headers = {
        "cookie": "supportWebp=true; newstatisticUUID=1716733315_1086707840; fu=1242577821; _csrfToken=yQltzgNj44UErVXluXQo0oDkcXXlzeJyBg1npj5F",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7",
        "Cookie": "supportWebp=true; supportwebp=true; _csrfToken=07cc5d3d-fb2b-4d08-8915-561aae1b4d86; qdrsnew=7%7C3%7C0%7C0%7C1; newstatisticUUID=1716733315_1086707840; fu=1597427940; traffic_search_engine=; _gid=GA1.2.592479407.1717706495; traffic_utm_referer=; se_ref=; Hm_lvt_f00f67093ce2f38f215010b699629083=1716387812,1716537935,1716983369,1717706496; _ga_FZMMH98S83=GS1.1.1717706494.30.1.1717706542.0.0.0; _ga=GA1.1.206926275.1716387806; _ga_PFYW0QLV3P=GS1.1.1717706494.30.1.1717706542.0.0.0; Hm_lpvt_f00f67093ce2f38f215010b699629083=1717706543",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    dictResponse = json.loads(response.text)
    footprintData = dictResponse['data']['historyData']

    return list(footprintData.values())

In [ ]:
df[]

In [65]:
userIds = ['216683708']
usersAll = []
for userId in userIds:
    userInfo = get_UserInfo(userId)
    userHistory = get_UserHistory(userId)
    userAll = userInfo + userHistory
    usersAll.append(userAll)

colnames = ['levelInfo','genderInfo','nameInfo','numberOfFollowers','numberOfFans','numberOfFavoriteBooks','bookshelfCollection','subscribedWorks',
            'rewardedWorks','monthlyVotes','recommendationVotes']
df = pd.DataFrame(usersAll,columns=colnames)
df

,levelInfo,genderInfo,nameInfo,numberOfFollowers,numberOfFans,numberOfFavoriteBooks,bookshelfCollection,subscribedWorks,rewardedWorks,monthlyVotes,recommendationVotes
0,Lv20,男,无,关注0,粉丝0,1,331,80,0,61,399


In [40]:


#<h2 class="user-title" data-l2="3"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Favorite Book List (1)</font></font></h2>

1

# SeleniumWire

This solution is not used.

In [ ]:
url = "https://www.qidian.com/chapter/1003692682/341658919/"

payload = ""
headers = {
    "cookie": "supportWebp=true",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Cookie": "newstatisticUUID=1713525319_1091593018; _csrfToken=s6jBkfoaACPlzf21q8bjniWywlm9SyZ1UseMewSI; fu=1861309847; supportWebp=true; _gid=GA1.2.1672002246.1713525321; Hm_lvt_f00f67093ce2f38f215010b699629083=1713525322; traffic_utm_referer=; trkf=1; supportwebp=true; _yep_uuid=35adbcb5-8d64-5043-c0ae-aef021d095cc; Hm_lpvt_f00f67093ce2f38f215010b699629083=1713525951; _ga_FZMMH98S83=GS1.1.1713525321.1.1.1713525951.0.0.0; _ga=GA1.1.597394370.1713525321; _ga_PFYW0QLV3P=GS1.1.1713525321.1.1.1713525951.0.0.0; w_tsfp=ltvgWVEE2utBvS0Q6Knslk6tFzA7Z2R7xFw0D+M9Os09A6YjVJ6A0oR+utfldCyCt5Mxutrd9MVxYnGCU9QjfBkWRM2Wb5tH1VPHx8NlntdKRQJtA5zVXVUXIb1x5TAVeW5WcEK0jmt9JodJzrBljF8Kuncg37ZlCa8hbMFbixsAqOPFm/97DxvSliPXAHGHM3wLc+6C6rgv8LlSgWyEtBu/eRlhAcxD0EWR0yEdCHAh9BPNd+1cMxmlKs+mTe9Gvy/hk2upNdLxiEox60I3sB49AtX02TXKL3ZEIAtrZViygr4ke66rNuYluTEZXL5TWwpN/FxC9qdk605dC3m5ZyGJBvty4QhSRftf/sr+fn+R0p7sIVpdu4wrkA0l9g=="
}

response = requests.request("GET", url, data=payload, headers=headers)
soup = bs(response.text,'html')

In [ ]:
h = soup.find_all('span')
h

In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip libvulkan1
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install chromedriver-autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,853 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By


# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()



'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/125/chromedriver'

In [ ]:
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
url = 'https://www.qidian.com/book/1010868264/'
driver.get(url)
driver.page_source

'<html><head><script>\n            var seqid = "971eae5eaeb9e49b017b926c1091e5f877de26f6a830b947b1aa476ad8d05293d083d6f84b6d0523c74c91f75f794c655fae0f3b6d2b33f1bd0159eb356bc8f9f13cefa9acdc1091927f840b3e018e7a"\n        </script>\n        <script src="https://ssl.captcha.qq.com/TCaptcha.js"></script></head></html>'

### Functions used to collect data

In [ ]:
def get_html_content(url,driver):
  driver.get(url)
  driver.refresh()
  driver.refresh()

  SCROLL_PAUSE_TIME = 2

  # Get scroll height
  last_height = driver.execute_script("return document.body.scrollHeight")

  book_list = []

  # does ~50 scroll downs
  while True:
      # Scroll down to bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height
  return driver.page_source


def get_book_list(content):
  soup = bs(content,'html')
  book_list = soup.find_all('li',{"class":"fl w50% bsbb pl20 pr20 pt8 pb8"})
  book_anchors = [book.find_all('a',{"class":"g_thumb _auto"})[0] for book in book_list]
  return book_anchors

def get_book_info(book):
  return [book['href'],book['title']]



### Collection Process
1. Generate the urls per book type
2. Collect data using selenium

In [ ]:
#novel_urls
novel_urls = ["https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=5"]
novel_genres = ['eastern-male','fantasy-male','horror-male','action-male','realistic-male','acg-male',
               'urban-male','games-male','scifi-male','sports-male','war-male','history-male','fantasy-female',
               'teen-female','scifi-female','urban-female','history-female','lgbt-female','general-female']
for g in novel_genres:
  for i in range(1,6):
    novel_urls.append(f"https://www.webnovel.com/stories/novel-{g}?bookStatus=0&orderBy={i}")


In [ ]:
#comic urls
comic_urls = ["https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=5"]
comic_genres = ['action','fantasy','harem','magic','eastern','scifi','sports','inspiring','comedy','drama',
'mystery','romance','lgbt','urban','transmigration','school','adventure','history','horror',
'wuxia','slice-of-life','cooking','diabolical']

for g in comic_genres:
  for i in range(1,6):
    comic_urls.append(f"https://www.webnovel.com/stories/comic-{g}?bookStatus=0&orderBy={i}")

In [ ]:
#fanfic_urls

fanfic_urls = ["https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=5"]
fanfic_genres = ['video-games','music-bands','book-literature','theater','anime-comics','celebrities','movies','tv','others']
for g in fanfic_genres:
  for i in range(1,6):
    fanfic_urls.append(f"https://www.webnovel.com/stories/fanfic-{g}?bookStatus=0&orderBy={i}")

In [ ]:
# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)
#url_list = novel_urls + comic_urls + fanfic_urls
book_info = []

for url in tqdm(fanfic_urls):
  html_content = get_html_content(url,driver)
  books = get_book_list(html_content)
  for book in books:
    book_info.append(get_book_info(book))

driver.close()

#save the information
#book_df = pd.DataFrame(book_info,columns=['Url','Title'])
#book_df.to_csv('webnovel_comic_list_all.csv',index=False)
#print(book_df.shape)
#book_df2 = book_df.drop_duplicates('Url')
#print(book_df2.shape)
#book_df2.to_csv('webnovel_comic_list.csv',index=False)

### Analyse the collected data

In [ ]:
#with duplicates
novels_all = pd.read_csv('data/webnovel_novel_list_all.csv')
comic_all = pd.read_csv('data/webnovel_comic_list_all.csv')
fanfic_all = pd.read_csv('data/webnovel_fanfic_list_all.csv')
#without duplicates
novels = pd.read_csv('data/webnovel_novel_list.csv')
comic = pd.read_csv('data/webnovel_comic_list.csv')
fanfic = pd.read_csv('data/webnovel_fanfic_list.csv')


In [ ]:
novels_all = novels_all.assign(Type='Novel')
novels = novels.assign(Type='Novel')
comic_all = comic_all.assign(Type='Comic')
comic = comic.assign(Type='Comic')
fanfic_all = fanfic_all.assign(Type='Fan-Fic')
fanfic = fanfic.assign(Type='Fan-Fic')

In [ ]:
webnovels_df = pd.concat([novels,comic,fanfic],ignore_index=True)
webnovels_df.drop_duplicates('Url')
webnovels_df.to_csv('webnovels.csv',index=False)
print(webnovels_df.shape)

(58997, 3)


In [ ]:
webnovels_df.head()

,Url,Title,Type
0,/book/supreme-magus_12820870105509205,Supreme Magus,Novel
1,/book/my-vampire-system_16709365405930105,My Vampire System,Novel
2,/book/blood-warlock-succubus-partner-in-the-ap...,Blood Warlock: Succubus Partner in the Apocalypse,Novel
3,/book/shadow-slave_22196546206090805,Shadow Slave,Novel
4,/book/under-the-oak-tree_21949582605391005,Under the Oak Tree,Novel


## Anotomy of a Single Book

In [ ]:
#thi is to extract tokens
book_list_df = book_list_df.replace(np.nan,'')
qidianPattern = 'book\/([0-9]*)\/'
webnovelPattern = '_([0-9]*)'
book_list_df['qidianBookId'] = book_list_df['qidianUrlcd '].apply(lambda x: re.findall(qidianPattern,x)[0] if re.findall(qidianPattern,x) else None)
book_list_df['webnovelBookId'] = book_list_df['webnovelUrl'].apply(lambda x: re.findall(webnovelPattern,x)[0] if re.findall(webnovelPattern,x) else None)

In [ ]:
def get_meta(book_url):
  response = requests.get(book_url,headers=headers)
  encoding = response.headers['Content-Type'].split('; ')[1].split('=')[-1]

  soup = bs(response.text,'html')

  body_text = bytes.decode(response.content, encoding)
  tree = html.fromstring(body_text)

  meta = soup.find_all('script', type='application/ld+json')[1]
  meta_dict = json.loads(meta.text.replace('\n','').replace('\r',''))[0]

  authorProfile = None
  if tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/address/h2/a"):
    authorProfile = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/address/h2/a")[0].get('href')

  if 'Chapters' in tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[1]/span")[0].text:
    numberOfChapters = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[1]/span")[0].text
  else:
    numberOfChapters = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text

  if 'Views' in tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text:
    numberOfViews = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text
  else:
    numberOfViews = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[3]/span")[0].text

  synopsis = soup.find_all('div',{'class':"j_synopsis"})[0].text

  return [meta_dict['name'],meta_dict['genre'],meta_dict['datePublished'],meta_dict['dateModified'],meta_dict['copyrightHolder'],meta_dict['description'],
                   meta_dict['aggregateRating']['bestRating'],meta_dict['aggregateRating']['ratingValue'],meta_dict['aggregateRating']['ratingCount'],meta_dict['author']['name'],
                   authorProfile,numberOfChapters,numberOfViews,synopsis]



In [ ]:
book_url = 'https://www.webnovel.com/book/reincarnation-of-the-strongest-sword-god_8527113906000305'
response = requests.get(book_url,headers=headers)
soup = bs(response.text,'html')
synopsis = soup.find_all('div',{'class':"j_synopsis"})


In [ ]:
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }
meta_column_names = ['webnovelId','webnovelName', 'webnovelGenre', 'webnovelDatePublished', 'webnovelDateModified', 'webnovelCopyrightHolder', 'webnovelDescription','webnovelBestRating',
                     'webnovelRatingValue','webnovelRatingCount','webnovelAuthor','webnovelAuthorProfile','webnovelNumberOfChapters','webnovelNumberOfViews','webnovelSynopsis']
meta_info = []
for i,row in tqdm(bookList.iterrows()):
  url = row['webnovelUrl']
  id = row['webnovelBookId']
  meta_info.append([id]+get_meta(url))



meta_df = pd.DataFrame(meta_info,columns=meta_column_names)

In [ ]:
meta_df.to_csv('webnovelMeta.csv',index=False)